In [2]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import datetime as dt
from random import seed
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
tf.compat.v1.experimental.output_all_intermediates(True)

In [3]:
tf.random.set_seed(36)

In [4]:
BASE_DIR_PATH = '/Users/neilb/Documents/dsci_thesis/Datasets'
DATASET_FILE = os.path.join(BASE_DIR_PATH, 'compiled_data_2016_2017.csv')

In [5]:
df = pd.read_csv(DATASET_FILE)
df

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420


In [6]:
df = df[['x', 't', 'Discharge_Sto_Nino', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']]
df.head(10)

,x,t,Discharge_Sto_Nino,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Velocity_Sto_Nino
0,14420,0.0,21.033407,0,1,2,0,0,12.18,0.026165
1,14420,3600.0,21.280072,0,1,1,1,0,12.19,0.026450
2,14420,7200.0,21.280072,1,1,1,0,1,12.19,0.026450
3,14420,10800.0,21.529056,0,0,0,1,0,12.20,0.026738
4,14420,14400.0,21.529056,1,1,1,0,0,12.20,0.026738
5,14420,18000.0,21.529056,0,0,1,0,0,12.20,0.026738
6,14420,21600.0,21.780375,0,1,1,0,0,12.21,0.027027
7,14420,25200.0,21.780375,0,0,0,0,0,12.21,0.027027
8,14420,28800.0,21.780375,1,0,1,0,0,12.21,0.027027
9,14420,32400.0,21.780375,0,0,0,0,0,12.21,0.027027


In [7]:
n = len(df)

In [8]:
df['friction_coeff'] = [0.033 for i in range(n)]
df['slope'] = [1/1500 for i in range(n)]
df.head()

C:\Users\neilb\AppData\Local\Temp\ipykernel_26112\1076625494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['friction_coeff'] = [0.033 for i in range(n)]
C:\Users\neilb\AppData\Local\Temp\ipykernel_26112\1076625494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['slope'] = [1/1500 for i in range(n)]


,x,t,Discharge_Sto_Nino,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Velocity_Sto_Nino,friction_coeff,slope
0,14420,0.0,21.033407,0,1,2,0,0,12.18,0.026165,0.033,0.000667
1,14420,3600.0,21.280072,0,1,1,1,0,12.19,0.026450,0.033,0.000667
2,14420,7200.0,21.280072,1,1,1,0,1,12.19,0.026450,0.033,0.000667
3,14420,10800.0,21.529056,0,0,0,1,0,12.20,0.026738,0.033,0.000667
4,14420,14400.0,21.529056,1,1,1,0,0,12.20,0.026738,0.033,0.000667


In [9]:
# Splitting for time series: split into 50-25-25
train_df = df[0:int(n*0.5)]
val_df = df[int(n*0.5):int(n*0.75)]
test_df = df[int(n*0.75):]

In [10]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    # Creating tf datasets for more convenient use and integration into model in the future
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [11]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        label_columns=['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']
    )

wide_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [1 2 3 4 5 6]
Label column name(s): ['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']

In [12]:
for example_inputs, example_labels in wide_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 12)
Labels shape (batch, time, features): (32, 6, 2)


In [39]:
wide_window.example[0][23]

<tf.Tensor: shape=(6, 12), dtype=float32, numpy=
array([[1.4420000e+04, 1.6268400e+07, 2.1780375e+01, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.2210000e+01, 2.7027491e-02, 3.3000000e-02, 6.6666666e-04],
       [1.4420000e+04, 1.6272000e+07, 2.2290085e+01, 5.0000000e+00,
        2.0000000e+00, 4.0000000e+00, 2.0000000e+00, 0.0000000e+00,
        1.2230000e+01, 2.7614763e-02, 3.3000000e-02, 6.6666666e-04],
       [1.4420000e+04, 1.6275600e+07, 2.4703785e+01, 3.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.2320000e+01, 3.0381477e-02, 3.3000000e-02, 6.6666666e-04],
       [1.4420000e+04, 1.6279200e+07, 3.0161154e+01, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 1.6000000e+01, 2.0000000e+00,
        1.2500000e+01, 3.6558975e-02, 3.3000000e-02, 6.6666666e-04],
       [1.4420000e+04, 1.6282800e+07, 3.3587364e+01, 2.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 2.0000000e+00, 2.0000000e

In [38]:
wide_window.example[1][23]

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[12.23      ,  0.02761476],
       [12.32      ,  0.03038148],
       [12.5       ,  0.03655897],
       [12.6       ,  0.04038884],
       [12.66      ,  0.04283196],
       [12.72      ,  0.04538836]], dtype=float32)>

In [13]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']
    )

conv_window

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): ['Waterlevel_Sto_Nino', 'Velocity_Sto_Nino']

In [35]:
for example_inputs, example_labels in conv_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 12)
Labels shape (batch, time, features): (32, 1, 2)


In [15]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [16]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

In [20]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

In [21]:
dense_history = compile_and_fit(dense, wide_window)

Epoch 1/20
274/274 [==============================] - 3s 6ms/step - loss: 6535821312.0000 - mean_squared_error: 6535826432.0000 - NSE: -169999344.0000 - r_square: 0.2337 - val_loss: 2288609.2500 - val_mean_squared_error: 2288608.7500 - val_NSE: -64235.5312 - val_r_square: 0.1647
Epoch 2/20
274/274 [==============================] - 1s 5ms/step - loss: 13752752.0000 - mean_squared_error: 13752752.0000 - NSE: -359040.7188 - r_square: 0.1846 - val_loss: 35129860.0000 - val_mean_squared_error: 35129868.0000 - val_NSE: -985860.4375 - val_r_square: 0.0194
Epoch 3/20
274/274 [==============================] - 1s 5ms/step - loss: 294218560.0000 - mean_squared_error: 294218528.0000 - NSE: -7662371.0000 - r_square: 0.0776 - val_loss: 312028704.0000 - val_mean_squared_error: 312028576.0000 - val_NSE: -8756373.0000 - val_r_square: 0.0074


In [22]:
dense_history.history

{'loss': [6535821312.0, 13752752.0, 294218560.0],
 'mean_squared_error': [6535826432.0, 13752752.0, 294218528.0],
 'NSE': [-169999344.0, -359040.71875, -7662371.0],
 'r_square': [0.23370963335037231, 0.1845589280128479, 0.07755687832832336],
 'val_loss': [2288609.25, 35129860.0, 312028704.0],
 'val_mean_squared_error': [2288608.75, 35129868.0, 312028576.0],
 'val_NSE': [-64235.53125, -985860.4375, -8756373.0],
 'val_r_square': [0.16474559903144836,
  0.019355759024620056,
  0.0074471174739301205]}

In [23]:
conv_history = compile_and_fit(conv_model, conv_window)

Epoch 1/20
274/274 [==============================] - 2s 4ms/step - loss: 7739751424.0000 - mean_squared_error: 7739751424.0000 - NSE: -199909232.0000 - r_square: 0.1219 - val_loss: 84456.4766 - val_mean_squared_error: 84456.4766 - val_NSE: -2370.1699 - val_r_square: 0.0274
Epoch 2/20
274/274 [==============================] - 1s 4ms/step - loss: 19759.2207 - mean_squared_error: 19759.2207 - NSE: -516.4610 - r_square: 0.0198 - val_loss: 27065.1074 - val_mean_squared_error: 27065.1074 - val_NSE: -758.5854 - val_r_square: 0.0055
Epoch 3/20
274/274 [==============================] - 1s 4ms/step - loss: 16408.7617 - mean_squared_error: 16408.7617 - NSE: -426.5931 - r_square: 0.0175 - val_loss: 35367.4805 - val_mean_squared_error: 35367.4805 - val_NSE: -992.1710 - val_r_square: 0.0089
Epoch 4/20
274/274 [==============================] - 1s 4ms/step - loss: 11874.0146 - mean_squared_error: 11874.0146 - NSE: -309.1353 - r_square: 0.0079 - val_loss: 25739.9883 - val_mean_squared_error: 25739.

In [24]:
lstm_history = compile_and_fit(lstm_model, wide_window)

Epoch 1/20
274/274 [==============================] - 5s 8ms/step - loss: 9.8152 - mean_squared_error: 9.8152 - NSE: 0.7442 - r_square: 3.0712e-04 - val_loss: 0.2068 - val_mean_squared_error: 0.2068 - val_NSE: 0.9942 - val_r_square: 8.1252e-15
Epoch 2/20
274/274 [==============================] - 2s 6ms/step - loss: 0.2802 - mean_squared_error: 0.2802 - NSE: 0.9927 - r_square: 9.7348e-05 - val_loss: 0.1796 - val_mean_squared_error: 0.1796 - val_NSE: 0.9950 - val_r_square: 7.8126e-15
Epoch 3/20
274/274 [==============================] - 2s 6ms/step - loss: 0.2390 - mean_squared_error: 0.2390 - NSE: 0.9938 - r_square: 9.9550e-05 - val_loss: 0.1302 - val_mean_squared_error: 0.1302 - val_NSE: 0.9963 - val_r_square: 1.4337e-14
Epoch 4/20
274/274 [==============================] - 2s 7ms/step - loss: 0.2338 - mean_squared_error: 0.2338 - NSE: 0.9939 - r_square: 1.3323e-04 - val_loss: 0.1550 - val_mean_squared_error: 0.1550 - val_NSE: 0.9956 - val_r_square: 1.1740e-14
Epoch 5/20
274/274 [====

In [25]:
val_performance = {}
performance = {}

In [26]:
val_performance['Dense'] = dense.evaluate(wide_window.val)

137/137 [==============================] - 0s 2ms/step - loss: 312028800.0000 - mean_squared_error: 312028608.0000 - NSE: -8757982.0000 - r_square: 0.0073


In [27]:
performance['Dense'] = dense.evaluate(wide_window.test, verbose=0)

In [28]:
val_performance['Conv'] = conv_model.evaluate(conv_window.val)

137/137 [==============================] - 0s 2ms/step - loss: 237854352.0000 - mean_squared_error: 237854352.0000 - NSE: -6674684.5000 - r_square: 0.2028


In [29]:
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [30]:
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)

137/137 [==============================] - 0s 3ms/step - loss: 0.1644 - mean_squared_error: 0.1644 - NSE: 0.9954 - r_square: 7.3664e-15


In [31]:
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [32]:
val_performance

{'Dense': [312028800.0, 312028608.0, -8757982.0, 0.007346741389483213],
 'Conv': [237854352.0, 237854352.0, -6674684.5, 0.20281143486499786],
 'LSTM': [0.16436277329921722,
  0.1643628031015396,
  0.9953826069831848,
  7.366373644694581e-15]}

In [33]:
performance

{'Dense': [593676416.0, 593676352.0, -14929997.0, 0.0017184509197250009],
 'Conv': [463176896.0, 463176896.0, -11647813.0, 0.04090144485235214],
 'LSTM': [0.1720539778470993,
  0.1720539629459381,
  0.9957028031349182,
  9.894492553573758e-15]}

In [34]:
print('test')

test
